In [14]:
import random
import tensorflow as tf

options = 4

# The following methods are used for randomized data generation

def generateKey(q):
    key = []
    for i in range(q):
        key.append(random.randint(1, options))
    return key

normal_accuracy = 0.80

def generateAns(key):
    answers = []
    for i in key:
        if (random.random() > normal_accuracy):
            answers.append(random.choice([n for n in range(1, options) if n != i]))
        else: 
            answers.append(i)
    return answers

# List contains a 1 or 0 depending on whether the key and ans list match
def generateAnsList(key, ans):
    ansL = []
    for i in range(len(key)):
        if (key[i] == ans[i]):
            ansL.append(1)
        else:
            ansL.append(0) 
    return ansL

def calcAcc(key, ans):
    ansL = generateAnsList(key, ans)
    return sum(ansL)/len(ansL)
    
def frameShift(ans, f):
    shiftedAns = ans.copy()
    a = 1
    while (a > f):
        shift = random.choice([n for n in range(-len(ans), len(ans)) if n != 0])
        if (shift < 0): 
            shift = -(shift + 1)
            shiftedAns.pop(shift)
            shiftedAns.append(random.randint(1, options))
        else: 
            shift = shift - 1
            shiftedAns.pop(-1)
            shiftedAns.insert(shift, random.randint(1, options))
        
        a = random.random()
        
    return shiftedAns

In [15]:
# Method that builds a neural network to predict if the input list has been shifted or not
# input_list must either contain 1 or 0s according to whether it matches the key

def detectFrameshift(input_list):
    q = len(input_list)
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(q,)),
        tf.keras.layers.Dense(q / 2, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'), 
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam',
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
    
    samples = q * 75
    shiftProb = 0.80


    ansInputs = []
    ansOutputs = []
    ansProbs = []

    for i in range(samples):
        key = generateKey(q)
        ansL = generateAnsList(key, generateAns(key))
        ansInputs.append(ansL)
        ansOutputs.append(0)
        ansProbs.append(calcAcc(key, ansL))
        ansL = generateAnsList(key, frameShift(generateAns(key), shiftProb))
        ansInputs.append(ansL)
        ansOutputs.append(1)
        ansProbs.append(calcAcc(key, ansL))   

    ansInputTrain = ansInputs[:int(samples*.8)]
    ansOutputTrain = ansOutputs[:int(samples*.8)]

    ansInputTest = ansInputs[int(samples*.8):]
    ansOutputTest = ansOutputs[int(samples*.8):]

    model.fit(ansInputTrain, ansOutputTrain, epochs=25)
    
    print(model.evaluate(ansInputTrain, ansOutputTrain))
    print(model.evaluate(ansInputTest, ansOutputTest))
    
    pred = model.predict([input_list])
    
    print(pred)
    
    return pred

In [16]:
# Method that builds a neural network to predict the initial accuracy of a frameshifted input list
# input_list must either contain 1 or 0s according to whether it matches the key list

def unshiftAccuracy(input_list):
    q = len(input_list)

    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(q,)),
        tf.keras.layers.Dense(q / 2, activation='relu'),
        tf.keras.layers.Dense(q / 2, activation='relu'),
        tf.keras.layers.Dense(q / 2, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='sgd',
        loss='mae',
        metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

    samples = q * 25
    shiftProb = 0.80


    ansInputs = []
    ansOutputs = []
    ansProbs = []

    for n in range(samples):
        for i in range(int(100/options),95,5):
            normal_accuracy = i/100
            key = generateKey(q)
            ansL = generateAnsList(key, frameShift(generateAns(key), shiftProb))
            ansInputs.append(ansL)
            ansOutputs.append(normal_accuracy)
            ansProbs.append(calcAcc(key, ansL))

    samples = len(ansInputs)

    ansInputTrain = ansInputs[:int(samples*.8)]
    ansOutputTrain = ansOutputs[:int(samples*.8)]


    ansInputTest = ansInputs[int(samples*.8):]
    ansOutputTest = ansOutputs[int(samples*.8):]

    print(len(ansInputTest))

    model.fit(ansInputTrain, ansOutputTrain, epochs=25)

    print(model.evaluate(ansInputTrain, ansOutputTrain))
    print(model.evaluate(ansInputTest, ansOutputTest))

    pred = model.predict([input_list])

    print(pred)

In [9]:
test1 = [1, 1, 1, 1, 1, 1, 1, 
        1, 1, 0, 0, 0, 0, 0, 0, 1]

test2 = [1, 1, 1, 1, 1, 1, 
          1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 
          1, 0, 1, 0, 0, 1, 0, 0]

In [10]:
detectFrameshift(test1)

Epoch 1/25
30/30 [==============================] - 2s 12ms/step - loss: 0.6714 - accuracy: 0.5510
Epoch 2/25
30/30 [==============================] - 0s 5ms/step - loss: 0.6469 - accuracy: 0.6031
Epoch 3/25
30/30 [==============================] - 0s 9ms/step - loss: 0.6272 - accuracy: 0.6469
Epoch 4/25
30/30 [==============================] - 0s 10ms/step - loss: 0.6042 - accuracy: 0.6979
Epoch 5/25
30/30 [==============================] - 0s 10ms/step - loss: 0.5766 - accuracy: 0.7354
Epoch 6/25
30/30 [==============================] - 0s 3ms/step - loss: 0.5389 - accuracy: 0.7740
Epoch 7/25
30/30 [==============================] - 0s 6ms/step - loss: 0.4905 - accuracy: 0.8073
Epoch 8/25
30/30 [==============================] - 0s 3ms/step - loss: 0.4491 - accuracy: 0.8292
Epoch 9/25
30/30 [==============================] - 0s 6ms/step - loss: 0.4145 - accuracy: 0.8396
Epoch 10/25
30/30 [==============================] - 0s 6ms/step - loss: 0.3903 - accuracy: 0.8490
Epoch 11/25
30/3

array([[0.96375537]], dtype=float32)

In [11]:
detectFrameshift(test2)

Epoch 1/25
57/57 [==============================] - 1s 4ms/step - loss: 0.6676 - accuracy: 0.5828
Epoch 2/25
57/57 [==============================] - 0s 3ms/step - loss: 0.5659 - accuracy: 0.7278
Epoch 3/25
57/57 [==============================] - 0s 3ms/step - loss: 0.4863 - accuracy: 0.8011
Epoch 4/25
57/57 [==============================] - 0s 3ms/step - loss: 0.4203 - accuracy: 0.8406
Epoch 5/25
57/57 [==============================] - 0s 4ms/step - loss: 0.3697 - accuracy: 0.8644
Epoch 6/25
57/57 [==============================] - 0s 5ms/step - loss: 0.3354 - accuracy: 0.8811
Epoch 7/25
57/57 [==============================] - 0s 4ms/step - loss: 0.3121 - accuracy: 0.8839
Epoch 8/25
57/57 [==============================] - 0s 4ms/step - loss: 0.2915 - accuracy: 0.8933
Epoch 9/25
57/57 [==============================] - 0s 4ms/step - loss: 0.2781 - accuracy: 0.8983
Epoch 10/25
57/57 [==============================] - 0s 4ms/step - loss: 0.2663 - accuracy: 0.9017
Epoch 11/25
57/57 [

array([[0.99470973]], dtype=float32)

In [12]:
unshiftAccuracy(test1)

1120
Epoch 1/25
140/140 [==============================] - 1s 3ms/step - loss: 0.1801 - mean_absolute_percentage_error: 34.8236
Epoch 2/25
140/140 [==============================] - 0s 3ms/step - loss: 0.1776 - mean_absolute_percentage_error: 35.5593
Epoch 3/25
140/140 [==============================] - 1s 5ms/step - loss: 0.1759 - mean_absolute_percentage_error: 36.2159
Epoch 4/25
140/140 [==============================] - 1s 4ms/step - loss: 0.1752 - mean_absolute_percentage_error: 36.6484
Epoch 5/25
140/140 [==============================] - 1s 4ms/step - loss: 0.1751 - mean_absolute_percentage_error: 36.8796
Epoch 6/25
140/140 [==============================] - 1s 4ms/step - loss: 0.1750 - mean_absolute_percentage_error: 37.0618
Epoch 7/25
140/140 [==============================] - 1s 4ms/step - loss: 0.1750 - mean_absolute_percentage_error: 37.1150
Epoch 8/25
140/140 [==============================] - 0s 3ms/step - loss: 0.1750 - mean_absolute_percentage_error: 37.2206
Epoch 9/25


In [13]:
unshiftAccuracy(test2)

2100
Epoch 1/25
263/263 [==============================] - 2s 4ms/step - loss: 0.1782 - mean_absolute_percentage_error: 37.4595
Epoch 2/25
263/263 [==============================] - 1s 3ms/step - loss: 0.1773 - mean_absolute_percentage_error: 37.6881
Epoch 3/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1768 - mean_absolute_percentage_error: 37.6977
Epoch 4/25
263/263 [==============================] - 1s 3ms/step - loss: 0.1765 - mean_absolute_percentage_error: 37.7205
Epoch 5/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1762 - mean_absolute_percentage_error: 37.6534
Epoch 6/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1760 - mean_absolute_percentage_error: 37.5449
Epoch 7/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1758 - mean_absolute_percentage_error: 37.7331
Epoch 8/25
263/263 [==============================] - 1s 3ms/step - loss: 0.1756 - mean_absolute_percentage_error: 37.5631
Epoch 9/25


In [18]:
input_list = test1

q = len(input_list)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(q,)),
    tf.keras.layers.Dense(q / 2, activation='relu'),
    tf.keras.layers.Dense(q / 2, activation='relu'),
    tf.keras.layers.Dense(q / 2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='sgd',
    loss='mae',
    metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

samples = q * 75
shiftProb = 0.80


ansInputs = []
ansOutputs = []
ansProbs = []

for n in range(samples):
    for i in range(int(100/options),95,5):
        normal_accuracy = i/100
        key = generateKey(q)
        ansL = generateAnsList(key, frameShift(generateAns(key), shiftProb))
        ansInputs.append(ansL)
        ansOutputs.append(normal_accuracy)
        ansProbs.append(calcAcc(key, ansL))

samples = len(ansInputs)

ansInputTrain = ansInputs[:int(samples*.8)]
ansOutputTrain = ansOutputs[:int(samples*.8)]


ansInputTest = ansInputs[int(samples*.8):]
ansOutputTest = ansOutputs[int(samples*.8):]

print(len(ansInputTest))

model.fit(ansInputTrain, ansOutputTrain, epochs=25)

print(model.evaluate(ansInputTrain, ansOutputTrain))
print(model.evaluate(ansInputTest, ansOutputTest))

pred = model.predict([input_list])

print(pred)

3360
Epoch 1/25
420/420 [==============================] - 2s 3ms/step - loss: 0.1605 - mean_absolute_percentage_error: 34.9462
Epoch 2/25
420/420 [==============================] - 1s 3ms/step - loss: 0.1526 - mean_absolute_percentage_error: 33.8766
Epoch 3/25
420/420 [==============================] - 1s 3ms/step - loss: 0.1476 - mean_absolute_percentage_error: 32.2014
Epoch 4/25
420/420 [==============================] - 1s 3ms/step - loss: 0.1438 - mean_absolute_percentage_error: 30.9294
Epoch 5/25
420/420 [==============================] - 2s 4ms/step - loss: 0.1410 - mean_absolute_percentage_error: 29.9977
Epoch 6/25
420/420 [==============================] - 2s 4ms/step - loss: 0.1389 - mean_absolute_percentage_error: 29.1943
Epoch 7/25
420/420 [==============================] - 1s 4ms/step - loss: 0.1373 - mean_absolute_percentage_error: 28.5664
Epoch 8/25
420/420 [==============================] - 2s 4ms/step - loss: 0.1362 - mean_absolute_percentage_error: 28.1547
Epoch 9/25


In [20]:
input_list = test2

q = len(input_list)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(q,)),
    tf.keras.layers.Dense(q / 2, activation='relu'),
    tf.keras.layers.Dense(q / 2, activation='relu'),
    tf.keras.layers.Dense(q / 2, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='sgd',
    loss='mae',
    metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])

samples = q * 25
shiftProb = 0.80


ansInputs = []
ansOutputs = []
ansProbs = []

for n in range(samples):
    for i in range(int(100/options),95,5):
        normal_accuracy = i/100
        key = generateKey(q)
        ansL = generateAnsList(key, frameShift(generateAns(key), shiftProb))
        ansInputs.append(ansL)
        ansOutputs.append(normal_accuracy)
        ansProbs.append(calcAcc(key, ansL))

samples = len(ansInputs)

ansInputTrain = ansInputs[:int(samples*.8)]
ansOutputTrain = ansOutputs[:int(samples*.8)]


ansInputTest = ansInputs[int(samples*.8):]
ansOutputTest = ansOutputs[int(samples*.8):]

print(len(ansInputTest))

model.fit(ansInputTrain, ansOutputTrain, epochs=25)

print(model.evaluate(ansInputTrain, ansOutputTrain))
print(model.evaluate(ansInputTest, ansOutputTest))

pred = model.predict([input_list])

print(pred)

2100
Epoch 1/25
263/263 [==============================] - 2s 5ms/step - loss: 0.1585 - mean_absolute_percentage_error: 36.9386
Epoch 2/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1473 - mean_absolute_percentage_error: 32.9954
Epoch 3/25
263/263 [==============================] - 1s 3ms/step - loss: 0.1414 - mean_absolute_percentage_error: 31.4480
Epoch 4/25
263/263 [==============================] - 1s 3ms/step - loss: 0.1364 - mean_absolute_percentage_error: 29.9514
Epoch 5/25
263/263 [==============================] - 1s 3ms/step - loss: 0.1322 - mean_absolute_percentage_error: 28.7381
Epoch 6/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1289 - mean_absolute_percentage_error: 27.5639
Epoch 7/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1263 - mean_absolute_percentage_error: 26.6765
Epoch 8/25
263/263 [==============================] - 1s 4ms/step - loss: 0.1243 - mean_absolute_percentage_error: 25.8995
Epoch 9/25
